In [ ]:
from pathlib import Path
import urllib.request as ureq
import os
import sys
from pyspark.sql import SparkSession, functions as F, types as T, Window

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.12:0.17.0 pyspark-shell'

spark = (
    SparkSession.builder
        .appName("L2")
        .getOrCreate()
)


In [ ]:
def download(url: str, fname: str) -> Path:
    path = Path(fname)
    if not path.exists():
        print(f"⇩  {fname}")
        ureq.urlretrieve(url, fname)
    return path

posts_xml  = download(
    "https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/posts_sample.xml",
    "posts_sample.xml"
)
langs_csv  = download(
    "https://git.ai.ssau.ru/tk/big_data/raw/branch/bachelor/data/programming-languages.csv",
    "programming-languages.csv"
)


⇩  posts_sample.xml
⇩  programming-languages.csv


In [ ]:
posts = (
    spark.read.format("xml")
        .option("rowTag", "row")
        .option("timestampFormat", "yyyy/MM/dd HH:mm:ss")
        .load(str(posts_xml))
)

languages = (
    spark.read.option("header", True).option("inferSchema", True)
        .csv(str(langs_csv))
        .dropna()
        .select("name")
        # «tag‑friendly» вид: только буквы/цифры/#+-
        .withColumn(
            "tag",
            F.lower(F.regexp_replace("name", r"[^A-Za-z0-9#+-]", ""))
        )
)


In [ ]:
posts_2010_20 = (
    posts.filter(F.col("_CreationDate")
                 .between("2010-01-01", "2020-12-31"))
          .select(
              F.year("_CreationDate").alias("year"),
              "_Tags"
          )
          .where("_Tags IS NOT NULL")
          # "<java><c#>" → ["java","c#"]
          .withColumn(
              "tag",
              F.explode(
                  F.split(
                      F.expr("substring(_Tags,2,length(_Tags)-2)"), "><"
                  )
              )
          )
          .select("year", F.lower("tag").alias("tag"))
)


In [6]:
lang_counts = (
    posts_2010_20
        .join(languages, "tag")              # inner‑join по полю tag
        .groupBy("year", "name")
        .count()
)

top10_per_year = (
    lang_counts
        .withColumn(
            "rn",
            F.row_number().over(
                Window.partitionBy("year").orderBy(F.desc("count"))
            )
        )
        .filter("rn <= 10")
        .orderBy("year", F.desc("count"))
        .select(
            F.col("year").alias("Year"),
            F.col("name").alias("Language"),
            F.col("count").alias("Count")
        )
)

top10_per_year.show(100, truncate=False)
top10_per_year.write.mode("overwrite").parquet("result_lang.parquet")


(
    top10_per_year
        .orderBy("Year", F.desc("Count"))  # внутри года — от популярного к редкому
        .write
        .mode("overwrite")
        .partitionBy("Year")               # отдельная папка на год: fast‑filter
        .option("compression", "snappy")   # дефолт Hadoop‑friendly компрессия
        .parquet("top10_languages_2010‑20.parquet")
)



+----+-----------+-----+
|Year|Language   |Count|
+----+-----------+-----+
|2010|Java       |52   |
|2010|PHP        |46   |
|2010|JavaScript |44   |
|2010|Python     |26   |
|2010|Objective-C|23   |
|2010|C          |20   |
|2010|Ruby       |12   |
|2010|Delphi     |8    |
|2010|S/SL       |4    |
|2010|R          |3    |
|2011|PHP        |102  |
|2011|Java       |93   |
|2011|JavaScript |83   |
|2011|Python     |37   |
|2011|Objective-C|34   |
|2011|C          |24   |
|2011|Ruby       |20   |
|2011|Perl       |9    |
|2011|Delphi     |8    |
|2011|Bash       |7    |
|2012|PHP        |154  |
|2012|JavaScript |132  |
|2012|Java       |124  |
|2012|Python     |69   |
|2012|Objective-C|45   |
|2012|C          |27   |
|2012|Ruby       |27   |
|2012|Bash       |10   |
|2012|R          |9    |
|2012|MATLAB     |6    |
|2013|PHP        |198  |
|2013|JavaScript |198  |
|2013|Java       |194  |
|2013|Python     |90   |
|2013|Objective-C|40   |
|2013|C          |36   |
|2013|Ruby       |32   |
